In [158]:
from copy import deepcopy

import torch
from torch import nn

In [159]:
torch.manual_seed(0);

In [160]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [161]:
from senmodel.model.utils import *

In [162]:
class SimpleFCN(nn.Module):
    def __init__(self, input_size=8):
        super(SimpleFCN, self).__init__()
        self.fc1 = nn.Linear(input_size, 4)
        self.fc2 = nn.Linear(4, 4)
        self.fc3 = nn.Linear(2, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = SimpleFCN()

In [163]:
sparse_model = convert_dense_to_sparse_network(model)

In [164]:
linear = nn.Linear(8, 1)

sparse_weight = dense_to_sparse(linear.weight.data)
sparse_bias = dense_to_sparse(linear.bias.data)
just_sparse_linear = ExpandingLinear(sparse_weight, sparse_bias)

sparse_linear = deepcopy(just_sparse_linear)

print(sparse_linear.weight_indices, "\n")
sparse_linear.replace(0, 6, 0)
sparse_linear.replace(0, 7, 0)
print(sparse_linear.weight_indices)
print(sparse_linear.embed_linears[0].weight_indices, "\n")

sparse_linear.replace(0, 8, 1)
sparse_linear.replace(0, 9, 1)
print(sparse_linear.weight_indices)
print(sparse_linear.embed_linears[1].weight_indices, "\n")

sparse_linear.replace(0, 11, 2)
print(sparse_linear.weight_indices)
print(sparse_linear.embed_linears[2].weight_indices, "\n")



tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 2, 3, 4, 5, 6, 7]]) 

tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 2, 3, 4, 5, 8, 9]])
tensor([[0, 1],
        [6, 7]]) 

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  1,  2,  3,  4,  5, 10, 11]])
tensor([[0, 1],
        [8, 9]]) 

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  1,  2,  3,  4,  5, 10, 11]])
tensor([[ 0],
        [11]]) 



In [165]:
from senmodel.metrics.nonlinearity_metrics import *


In [166]:
criterion = nn.MSELoss()

metrics = [
    GradientMeanEdgeMetric(criterion),
    PerturbationSensitivityEdgeMetric(criterion),
]

In [167]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression

X, y = make_regression(n_samples=1000, n_features=8, n_informative=8, random_state=42)
X = torch.from_numpy(X).float()
y = torch.from_numpy(y).float()

dataset = list(zip(X, y))
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [168]:

from senmodel.metrics.edge_finder import *
ef = EdgeFinder(metrics[0], test_loader, device=device)

In [169]:


print("values:", ef.calculate_edge_metric_for_dataloader(sparse_linear))
chosen_edges = ef.choose_edges_top_k(sparse_linear, 4)
print("choose:", chosen_edges)

chosen_edges = ef.choose_edges_top_percent(sparse_linear, 0.5)
print("choose:", chosen_edges)

chosen_edges = ef.choose_edges_threshold(sparse_linear, 30)
print("choose:", chosen_edges)

values: tensor([212.2146,  97.8833,  80.0896, 174.4840,  73.9763,  52.1134,   3.2056,
         17.0417])
choose: tensor([[0, 0, 0, 0],
        [0, 3, 1, 2]])
choose: tensor([[0, 0, 0, 0],
        [0, 3, 1, 2]])
choose: tensor([[0, 0, 0, 0, 0, 0],
        [0, 1, 2, 3, 4, 5]])


In [170]:
sparse_linear.weight_indices

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  1,  2,  3,  4,  5, 10, 11]])

In [171]:
sparse_linear.replace_many(*chosen_edges, 3)

In [172]:
sparse_linear.weight_indices

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0],
        [10, 11, 12, 13, 14, 15, 16, 17]])

In [173]:
print("values:", ef.calculate_edge_metric_for_dataloader(sparse_linear))
chosen_edges = ef.choose_edges_top_k(sparse_linear, 5)
print("choose:", chosen_edges)

values: tensor([ 3.1923, 16.9779,  8.7249, 51.5558,  5.7307,  0.9609, 21.5173, 31.2697])
choose: tensor([[ 0,  0,  0,  0,  0],
        [13, 17, 16, 11, 12]])


In [174]:
sparse_linear.replace_many(*chosen_edges, 4)

In [175]:
sparse_linear.weight_indices

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0],
        [10, 14, 15, 18, 19, 20, 21, 22]])

In [176]:
# print("values:", ef.calculate_edge_metric_for_dataloader(sparse_linear))
# chosen_edges = ef.choose_edges_top_k(sparse_linear, 3)
# print("choose:", chosen_edges)

In [177]:
sparse_linear.replace_many(*chosen_edges, 2)